<a href="https://colab.research.google.com/github/Nili3005/ML/blob/main/Project_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install --upgrade ngboost
!pip install ucimlrepo

# Usage

We'll start with a probabilistic regression example on the Boston housing dataset:

In [20]:
import sys
sys.path.append('/Users/c242587/Desktop/projects/git/ngboost')

In [21]:
from ngboost import NGBRegressor

import pandas as pd
import numpy as np

#from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#X, Y = load_boston(True)
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
Y = raw_df.values[1::2, 2]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

ngb = NGBRegressor().fit(X_train, Y_train)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)

[iter 0] loss=3.6209 val_loss=0.0000 scale=1.0000 norm=6.6042
[iter 100] loss=2.6762 val_loss=0.0000 scale=2.0000 norm=4.8248
[iter 200] loss=2.1203 val_loss=0.0000 scale=2.0000 norm=3.3354
[iter 300] loss=1.8791 val_loss=0.0000 scale=2.0000 norm=2.9357
[iter 400] loss=1.7506 val_loss=0.0000 scale=1.0000 norm=1.3613
Test MSE 12.883412481764202
Test NLL 4.862626385619798


Getting the estimated distributional parameters at a set of points is easy. This returns the predicted mean and standard deviation of the first five observations in the test set:

In [22]:
Y_dists[0:5].params

{'loc': array([42.38736532, 25.9625905 , 19.71686918, 36.42981987, 20.7587038 ]),
 'scale': array([1.91300855, 0.63491177, 1.42851485, 1.08972218, 1.2102706 ])}

## Distributions

NGBoost can be used with a variety of distributions, broken down into those for regression (support on an infinite set) and those for classification (support on a finite set).

### Regression Distributions

| Distribution | Parameters | Implemented Scores | Reference |
| --- | --- | --- | --- |
| `Normal` | `loc`, `scale` | `LogScore`, `CRPScore` | [`scipy.stats` normal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) |
| `LogNormal` | `s`, `scale` | `LogScore`, `CRPScore` | [`scipy.stats` lognormal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html) |
| `Exponential` | `scale` | `LogScore`, `CRPScore` | [`scipy.stats` exponential](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html) |

Regression distributions can be used through the `NGBRegressor()` constructor by passing the appropriate class as the `Dist` argument. `Normal` is the default.

In [23]:
from ngboost.distns import Exponential, Normal

#X, Y = load_boston(True)
X_reg_train, X_reg_test, Y_reg_train, Y_reg_test = train_test_split(X, Y, test_size=0.2)

ngb_norm = NGBRegressor(Dist=Normal, verbose=False).fit(X_reg_train, Y_reg_train)
ngb_exp = NGBRegressor(Dist=Exponential, verbose=False).fit(X_reg_train, Y_reg_train)

There are two prediction methods for `NGBRegressor` objects: `predict()`, which returns point predictions as one would expect from a standard regressor, and `pred_dist()`, which returns a distribution object representing the conditional distribution of $Y|X=x_i$ at the points $x_i$ in the test set.

In [24]:
ngb_norm.predict(X_reg_test)[0:5]

array([18.87245887, 31.84435508, 17.14666837, 11.1388688 , 19.33673201])

In [25]:
ngb_exp.predict(X_reg_test)[0:5]

array([18.59998241, 29.42295241, 17.03024975, 10.27697713, 19.13234761])

In [26]:
ngb_exp.pred_dist(X_reg_test)[0:5].params

{'scale': array([18.59998241, 29.42295241, 17.03024975, 10.27697713, 19.13234761])}

#### Survival Regression

NGBoost supports analyses of right-censored data. Any distribution that can be used for regression in NGBoost can also be used for survival analysis in theory, but this requires the implementation of the right-censored version of the appropriate score. At the moment, `LogNormal` and `Exponential` have these scores implemented. To do survival analysis, use `NGBSurvival` and pass both the time-to-event (or censoring) and event indicator vectors to  `fit()`:

In [27]:
import numpy as np
from ngboost import NGBSurvival
from ngboost.distns import LogNormal

#X, Y = load_boston(True)
X_surv_train, X_surv_test, Y_surv_train, Y_surv_test = train_test_split(X, Y, test_size=0.2)

# introduce administrative censoring to simulate survival data
T_surv_train = np.minimum(Y_surv_train, 30) # time of an event or censoring
E_surv_train = Y_surv_train > 30 # 1 if T[i] is the time of an event, 0 if it's a time of censoring

ngb = NGBSurvival(Dist=LogNormal).fit(X_surv_train, T_surv_train, E_surv_train)

[iter 0] loss=1.2481 val_loss=0.0000 scale=4.0000 norm=2.4305
[iter 100] loss=0.5315 val_loss=0.0000 scale=2.0000 norm=0.8511
[iter 200] loss=0.2873 val_loss=0.0000 scale=2.0000 norm=0.5092
[iter 300] loss=0.1175 val_loss=0.0000 scale=4.0000 norm=0.5194
[iter 400] loss=-0.0074 val_loss=0.0000 scale=2.0000 norm=0.1994


The scores currently implemented assume that the censoring is independent of survival, conditional on the observed predictors.

### Classification Distributions

| Distribution | Parameters | Implemented Scores | Reference |
| --- | --- | --- | --- |
| `k_categorical(K)` | `p0`, `p1`... `p{K-1}` | `LogScore` | [Categorical distribution on Wikipedia](https://en.wikipedia.org/wiki/Categorical_distribution) |
| `Bernoulli` | `p` | `LogScore` | [Bernoulli distribution on Wikipedia](https://en.wikipedia.org/wiki/Bernoulli_distribution) |

Classification distributions can be used through the `NGBClassifier()` constructor by passing the appropriate class as the `Dist` argument. `Bernoulli` is the default and is equivalent to `k_categorical(2)`.

In [28]:
from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli
from sklearn.datasets import load_breast_cancer

#X, y = load_breast_cancer(True)
data = load_breast_cancer()
X, y = data.data, data.target
y[0:15] = 2 # artificially make this a 3-class problem instead of a 2-class problem
X_cls_train, X_cls_test, Y_cls_train, Y_cls_test  = train_test_split(X, y, test_size=0.2)

ngb_cat = NGBClassifier(Dist=k_categorical(3), verbose=False) # tell ngboost that there are 3 possible outcomes
_ = ngb_cat.fit(X_cls_train, Y_cls_train) # Y should have only 3 values: {0,1,2}

When using NGBoost for classification, the outcome vector `Y` must consist only of integers from 0 to K-1, where K is the total number of classes. This is consistent with the classification standards in sklearn.

`NGBClassifier` objects have three prediction methods: `predict()` returns the most likely class, `predict_proba()` returns the class probabilities, and `pred_dist()` returns the distribution object.

In [29]:
ngb_cat.predict(X_cls_test)[0:5]

array([1, 1, 1, 0, 1])

In [30]:
ngb_cat.predict_proba(X_cls_test)[0:5]

array([[5.37159831e-03, 9.94220510e-01, 4.07892072e-04],
       [5.37159831e-03, 9.94220510e-01, 4.07892072e-04],
       [5.69442338e-02, 9.39828960e-01, 3.22680569e-03],
       [9.74308461e-01, 9.16630937e-03, 1.65252292e-02],
       [5.37159831e-03, 9.94220510e-01, 4.07892072e-04]])

In [31]:
ngb_cat.pred_dist(X_cls_test)[0:5].params

{'p0': array([0.0053716 , 0.0053716 , 0.05694423, 0.97430846, 0.0053716 ]),
 'p1': array([0.99422051, 0.99422051, 0.93982896, 0.00916631, 0.99422051]),
 'p2': array([0.00040789, 0.00040789, 0.00322681, 0.01652523, 0.00040789])}

## Scores

NGBoost supports the log score (`LogScore`, also known as negative log-likelihood) and CRPS (`CRPScore`), although each score may not be implemented for each distribution. The score is specified by the `Score` argument in the constructor.

In [32]:
from ngboost.scores import LogScore, CRPScore

NGBRegressor(Dist=Exponential, Score=CRPScore, verbose=False).fit(X_reg_train, Y_reg_train)
NGBClassifier(Dist=k_categorical(3), Score=LogScore, verbose=False).fit(X_cls_train, Y_cls_train)

NGBClassifier(random_state=RandomState(MT19937) at 0x780F8477F940,
              verbose=False)

## Base Learners

NGBoost can be used with any sklearn regressor as the base learner, specified with the `Base` argument. The default is a depth-3 regression tree.

In [33]:
from sklearn.tree import DecisionTreeRegressor

learner = DecisionTreeRegressor(criterion='friedman_mse', max_depth=5)

NGBSurvival(Dist=Exponential, Score=CRPScore, Base=learner, verbose=False).fit(X_surv_train, T_surv_train, E_surv_train)

NGBSurvival(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=5),
            Dist=<class 'ngboost.distns.utils.SurvivalDistnClass.<locals>.SurvivalDistn'>,
            Score=<class 'ngboost.scores.CRPScore'>,
            random_state=RandomState(MT19937) at 0x780F8477F940, verbose=False)

## Other Arguments

The learning rate, number of estimators, minibatch fraction, and column subsampling are also easily adjusted:

In [34]:
ngb = NGBRegressor(n_estimators=100, learning_rate=0.01,
             minibatch_frac=0.5, col_sample=0.5)
ngb.fit(X_reg_train, Y_reg_train)

[iter 0] loss=3.5244 val_loss=0.0000 scale=2.0000 norm=12.0344


NGBRegressor(col_sample=0.5, minibatch_frac=0.5, n_estimators=100,
             random_state=RandomState(MT19937) at 0x780F8477F940)

Sample weights (for training) are set using the `sample_weight` argument to `fit`.

In [35]:
ngb = NGBRegressor(n_estimators=100, learning_rate=0.01,
             minibatch_frac=0.5, col_sample=0.5)
weights = np.random.random(Y_reg_train.shape)
ngb.fit(X_reg_train, Y_reg_train, sample_weight=weights)

[iter 0] loss=3.5543 val_loss=0.0000 scale=1.0000 norm=6.0135


NGBRegressor(col_sample=0.5, minibatch_frac=0.5, n_estimators=100,
             random_state=RandomState(MT19937) at 0x780F8477F940)